# Verificando os dados extraídos

In [1]:
from context import fakenews
from fakenews import preprocess as pre

sentences, labels = pre.run()
print(sentences[0][:10])

FileNotFoundError: [Errno 2] No such file or directory: '/home/thalesaguiar/Dev/FakeNews/notebooks/data/Fake.csv'